## Code Review 1
## Exercise 2: Non-Derivable Itemsets

In [1]:
import itertools

#### Set up files

In [2]:
'''
function to create dictionary from itemsets/supports file
output is dictionary with key: itemsets as frozen sets and value: support
'''

def create_dict_from_file(filename):
    f = open(filename, 'r')
    d = {}
    # for each line in f
    for lines in f:
        # key and value splits on -
        k,v = [j.rstrip("\n") for j in lines.split(' - ') if j != '\n']
        # create key 
        fznset = frozenset(k.split(' '))
        # update dictionary with the frozen set of itemsets as key, and value as supports
        d[fznset] = int(v)
    return d

In [3]:
'''
function to create itemsets from ndi file
output is list of sets for each itemset
'''

def create_itemsets(filename):
    f = open(filename, 'r')
    # create empty list
    itemsets = []
    # for each line in file
    for line in f:
        # strip whitepsace
        lines = [j.rstrip("\n") for j in line.split(' ') if j != '\n']
        # append each line as a set to the list
        itemsets.append(set(lines))    
    
    return itemsets

#### Create powerset function

In [4]:
'''
function to get all subsets of a itemset
output is a set of all sets
takes one itemset (single_set)
'''

def get_powersets(single_set):
    # create empty list
    ps_list = []
    # for each element in the itemset, create its combinations subset for the length of the itemset
    sub = sum([list(map(list, itertools.combinations(single_set, i))) for i in range(len(single_set) + 1)], [])
    # for each element in the subsets
    for j in sub:
        # add it to the ps_list as a frozenset
        ps_list.append(frozenset(j))
    # create ps as a set of ps_list
    ps = set(ps_list)
    return ps    

#### Compute bound function

In [5]:
'''
function to compute bound value
output is the bound value for that current subset
takes current itemset working on (i), current subset in the itemset (c), current element in the current subset (s), and suports
'''

def compute_bound(cur_itemset, cur_subset, sub_sets, supports):
    # set bound_val to 0
    bound_val = 0
    # for each element (s) in current subset (c)
    for s in sub_sets:
        # if current subset (c) is a subset of s. if it is not a subset, we do nothing, as we move upwards in the subset lattice
        # ex: if current subset (c) is 'AB' and we are working towards 'ABC' and (s) is 'A', we skip 'A' as it 'AB' is not a subset of 'A'
        # you can be a subset of yourself. 'AB' is a subset of 'AB'
        # empty set is a subset of all subsets
        if cur_subset.issubset(s):
            # if s is empty set
            if s == set(): 
                # sum all support values in file, as they everything that is not the set
                empty = sum(supports.values()) 
                # compute coef if the support is a subtraction or addtion in the support lattic
                # length of current itemset - length of s
                # ex: len(AB) - len(A) = 1, -1^2 = 1
                coef = (-1)**(len(cur_itemset.difference(s)) + 1)
                # add to bound value coef x empty support val
                bound_val += (coef*empty)
            # or if current subset(c) is the same as current itemset (i) (this is the last value at the top of the lattice)
            elif c == cur_itemset:
                # get the support of s, do not calculate coef as it will be positive
                sup_s = supports.get(s)
                # add the support to bound value
                bound_val += sup_s
            # else if none of the above, and c is subset of s, 
            else:
                # get the support of s
                sup_s = supports.get(s)
                # calculate coeficient
                coef = (-1) ** (len(cur_itemset.difference(s)) + 1)
                # multiply coef X support and add to bound value
                bound_val += (coef*sup_s)
    return bound_val

#### Evaluate results

In [6]:
'''
function to evaluate the upper and lower bounds
output is the max lower bound and min upper bound and if it is derivable or not
takes current itemset (i), lower_bound, upper_bound
'''

def results (i, lower_bound, upper_bound):
    # if the max lower is the same as the min upper
    if max(lower_bound)==min(upper_bound):
        # it is derivable
        deriv = 'derivable'
    # else it is not derivable
    else:
        deriv = 'non-derivable'
    results = '{}: \t [lb: {}] \t [ub: {}] \t {} \n'.format(i, max(lower_bound), min(upper_bound), deriv)
    return results

#### Main

In [7]:
supports = create_dict_from_file('itemsets.txt')
itemsets = create_itemsets('ndi.txt')

# for each itemset in the ndi file
for i in itemsets:
    lower_bound = {0}
    upper_bound = set({})
    # call get_powersets function to get all subsets of the current itemset in the file
    sub_set = get_powersets(i)
    # for each subset (c) in the powersets for the current itemset (i)
    for c in sub_set:
        # if the length of the current item set (i) without the current subset (c) is even 
        if len(i.difference(c))%2 == 0:
            # compute lower bound value
            low = compute_bound(i, c, sub_set, supports)
            # add lower bound value to lower_bound list
            lower_bound.add(low)
        # else if it is odd
        else: 
            # compute upper bound value
            up = compute_bound(i, c, sub_set, supports)
            # add to upper_bound list
            upper_bound.add(up)
               
    
    print(results(i, lower_bound, upper_bound))
    

{'52', '40', '29', '34', '62'}: 	 [lb: 2888] 	 [ub: 0] 	 non-derivable 

{'29', '7'}: 	 [lb: 3069] 	 [ub: 7] 	 non-derivable 

{'48', '29', '58'}: 	 [lb: 2997] 	 [ub: 1] 	 non-derivable 

{'52', '60', '40', '29', '36', '58', '7'}: 	 [lb: 2890] 	 [ub: 0] 	 non-derivable 

{'52', '5', '60', '40'}: 	 [lb: 2893] 	 [ub: 0] 	 non-derivable 

{'58', '36', '40', '7'}: 	 [lb: 2952] 	 [ub: 0] 	 non-derivable 

{'52', '66', '60', '40', '36', '58'}: 	 [lb: 2888] 	 [ub: 0] 	 non-derivable 

